In [5]:
import requests
import time
import pandas as pd
from datetime import datetime

# Your Riot API key
api_key = 'RGAPI-25f46fad-7ece-43e8-a13d-1351ed49024a'

# Base URLs for API calls
riot_base_url = 'https://europe.api.riotgames.com'
lol_base_url = 'https://eun1.api.riotgames.com'

# Rate limiting constants
requests_per_second = 20
requests_per_minute = 100

# Initialize time tracking for rate limiting
start_time = time.time()
requests_count = 0

# Example list of summoners
summoner_list = [
    {'gameName': 'Ashley Graves', 'tagLine': '000'}, {'gameName': 'Δ Desperate', 'tagLine': 'adi'}, {'gameName': 'acelosss', 'tagLine': '12345'},{'gameName': 'LunaTown', 'tagLine': 'EUNE'}, {'gameName': 'noobslayer69420', 'tagLine': 'noob'}
]

# Date range for matches (timestamps in milliseconds)
start_date = int(datetime(2023, 9, 1).timestamp() * 1000)
end_date = int(datetime(2024, 9, 1).timestamp() * 1000)

# Initialize lists to hold the data
filtered_match_data = []

def handle_rate_limiting():
    global requests_count, start_time
    elapsed_time = time.time() - start_time

    if requests_count >= requests_per_second:
        time.sleep(1)
        requests_count = 0

    if elapsed_time < 60 and requests_count >= requests_per_minute:
        time.sleep(60 - elapsed_time)
        start_time = time.time()
        requests_count = 0

def get_puuid(game_name, tag_line):
    global requests_count
    url = f'{riot_base_url}/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print(f"Error fetching PUUID for {game_name}#{tag_line}: {response.status_code} {response.text}")
        return None

def get_match_history(puuid):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match history for PUUID {puuid}: {response.status_code} {response.text}")
        return []

def get_filtered_match_details(matchId):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/{matchId}?api_key={api_key}'  # Updated endpoint to get match details
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        match_data = response.json()

        # Check if the match falls within the specified date range
        game_creation = match_data['info']['gameCreation']
        if not (start_date <= game_creation <= end_date):
            return None

        # Extracting relevant data based on the given requirements
        for participant in match_data['info']['participants']:
            match_details = {
                'summonerName': participant['summonerName'],
                'championId': participant['championId'],
                'role': participant['teamPosition'],
                'kills': participant['kills'],
                'deaths': participant['deaths'],
                'assists': participant['assists'],
                'totalDamageDealt': participant['totalDamageDealtToChampions'],
                'totalDamageTaken': participant['totalDamageTaken'],
                'goldEarned': participant['goldEarned'],
                'totalMinionsKilled': participant['totalMinionsKilled'],
            }

            # Add team performance data (objectives)
            for team in match_data['info']['teams']:
                if participant['teamId'] == team['teamId']:
                    match_details.update({
                        'win': team['win'],
                        'dragons': team['objectives']['dragon']['kills'],
                        'barons': team['objectives']['baron']['kills'],
                        'turretsKilled': team['objectives']['tower']['kills']
                    })

            # Append the filtered data for each participant
            filtered_match_data.append(match_details)
    else:
        print(f"Error fetching match details for match ID {matchId}: {response.status_code} {response.text}")
        return None

# Process each summoner in the list
for summoner in summoner_list:
    puuid = get_puuid(summoner['gameName'], summoner['tagLine'])
    if puuid:
        matchIds = get_match_history(puuid)
        for matchId in matchIds:
            get_filtered_match_details(matchId)

# Convert the list to a DataFrame
filtered_match_df = pd.DataFrame(filtered_match_data)

# Display the DataFrame (for verification purposes)
print("\nFiltered Match Data:")
print(filtered_match_df.head())

# Save DataFrame to CSV
#filtered_match_df.to_csv('filtered_match_data.csv', index=False)

print("Data collection complete.")


Error fetching match details for match ID EUN1_3649385804: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match details for match ID EUN1_3649341240: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching PUUID for Δ Desperate#adi: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching PUUID for acelosss#12345: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching PUUID for LunaTown#EUNE: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching PUUID for noobslayer69420#noob: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}

Filtered Match Data:
       summonerName  championId     role  kills  deaths  assists  \
0     GirlyisTeepee          92      TOP      3       4        6   
1  T1 VIPERGUMAYUZI         887   JUNGLE      7       2        4   
2     PridefulQueen          99   MIDDLE      4       3        5   
3      Chef He3S

In [7]:
filtered_match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   summonerName        730 non-null    object
 1   championId          730 non-null    int64 
 2   role                730 non-null    object
 3   kills               730 non-null    int64 
 4   deaths              730 non-null    int64 
 5   assists             730 non-null    int64 
 6   totalDamageDealt    730 non-null    int64 
 7   totalDamageTaken    730 non-null    int64 
 8   goldEarned          730 non-null    int64 
 9   totalMinionsKilled  730 non-null    int64 
 10  win                 730 non-null    bool  
 11  dragons             730 non-null    int64 
 12  barons              730 non-null    int64 
 13  turretsKilled       730 non-null    int64 
dtypes: bool(1), int64(11), object(2)
memory usage: 75.0+ KB


In [8]:
filtered_match_df.to_csv('Summoner_1_filtered_match_data.csv', index=False)

In [9]:
import requests
import time
import pandas as pd
from datetime import datetime

# Your Riot API key
api_key = 'RGAPI-25f46fad-7ece-43e8-a13d-1351ed49024a'

# Base URLs for API calls
riot_base_url = 'https://europe.api.riotgames.com'
lol_base_url = 'https://eun1.api.riotgames.com'

# Rate limiting constants
requests_per_second = 20
requests_per_minute = 100

# Initialize time tracking for rate limiting
start_time = time.time()
requests_count = 0

# Example list of summoners
summoner_list = [
    {'gameName': 'Δ Desperate', 'tagLine': 'adi'}
]

# Date range for matches (timestamps in milliseconds)
start_date = int(datetime(2023, 9, 1).timestamp() * 1000)
end_date = int(datetime(2024, 9, 1).timestamp() * 1000)

# Initialize lists to hold the data
filtered_match_data = []

def handle_rate_limiting():
    global requests_count, start_time
    elapsed_time = time.time() - start_time

    if requests_count >= requests_per_second:
        time.sleep(1)
        requests_count = 0

    if elapsed_time < 60 and requests_count >= requests_per_minute:
        time.sleep(60 - elapsed_time)
        start_time = time.time()
        requests_count = 0

def get_puuid(game_name, tag_line):
    global requests_count
    url = f'{riot_base_url}/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print(f"Error fetching PUUID for {game_name}#{tag_line}: {response.status_code} {response.text}")
        return None

def get_match_history(puuid):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match history for PUUID {puuid}: {response.status_code} {response.text}")
        return []

def get_filtered_match_details(matchId):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/{matchId}?api_key={api_key}'  # Updated endpoint to get match details
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        match_data = response.json()

        # Check if the match falls within the specified date range
        game_creation = match_data['info']['gameCreation']
        if not (start_date <= game_creation <= end_date):
            return None

        # Extracting relevant data based on the given requirements
        for participant in match_data['info']['participants']:
            match_details = {
                'summonerName': participant['summonerName'],
                'championId': participant['championId'],
                'role': participant['teamPosition'],
                'kills': participant['kills'],
                'deaths': participant['deaths'],
                'assists': participant['assists'],
                'totalDamageDealt': participant['totalDamageDealtToChampions'],
                'totalDamageTaken': participant['totalDamageTaken'],
                'goldEarned': participant['goldEarned'],
                'totalMinionsKilled': participant['totalMinionsKilled'],
            }

            # Add team performance data (objectives)
            for team in match_data['info']['teams']:
                if participant['teamId'] == team['teamId']:
                    match_details.update({
                        'win': team['win'],
                        'dragons': team['objectives']['dragon']['kills'],
                        'barons': team['objectives']['baron']['kills'],
                        'turretsKilled': team['objectives']['tower']['kills']
                    })

            # Append the filtered data for each participant
            filtered_match_data.append(match_details)
    else:
        print(f"Error fetching match details for match ID {matchId}: {response.status_code} {response.text}")
        return None

# Process each summoner in the list
for summoner in summoner_list:
    puuid = get_puuid(summoner['gameName'], summoner['tagLine'])
    if puuid:
        matchIds = get_match_history(puuid)
        for matchId in matchIds:
            get_filtered_match_details(matchId)

# Convert the list to a DataFrame
filtered_match_df = pd.DataFrame(filtered_match_data)

# Display the DataFrame (for verification purposes)
print("\nFiltered Match Data:")
print(filtered_match_df.head())

# Save DataFrame to CSV
#filtered_match_df.to_csv('filtered_match_data.csv', index=False)

print("Data collection complete.")

Error fetching match details for match ID EUN1_3623610412: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match details for match ID EUN1_3623291559: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}

Filtered Match Data:
    summonerName  championId     role  kills  deaths  assists  \
0     zNRekTuSeu          90      TOP      1      11        5   
1   ŠnorchlBokem         427   JUNGLE      3       8       12   
2       Ayndalyn         711   MIDDLE      8      13        3   
3  In same World         236   BOTTOM     10       6        3   
4    DDjednasedm         526  UTILITY      1       6       13   

   totalDamageDealt  totalDamageTaken  goldEarned  totalMinionsKilled    win  \
0             19541             21171        7522                 148  False   
1              8887             17987        8855                   9  False   
2             26545             25537        9365                 107  False   
3             2

In [10]:
filtered_match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   summonerName        698 non-null    object 
 1   championId          698 non-null    int64  
 2   role                698 non-null    object 
 3   kills               698 non-null    int64  
 4   deaths              698 non-null    int64  
 5   assists             698 non-null    int64  
 6   totalDamageDealt    698 non-null    int64  
 7   totalDamageTaken    698 non-null    int64  
 8   goldEarned          698 non-null    int64  
 9   totalMinionsKilled  698 non-null    int64  
 10  win                 674 non-null    object 
 11  dragons             674 non-null    float64
 12  barons              674 non-null    float64
 13  turretsKilled       674 non-null    float64
dtypes: float64(3), int64(8), object(3)
memory usage: 76.5+ KB


In [11]:
filtered_match_df.to_csv('Summoner_2_filtered_match_data.csv', index=False)

In [12]:
import requests
import time
import pandas as pd
from datetime import datetime

# Your Riot API key
api_key = 'RGAPI-25f46fad-7ece-43e8-a13d-1351ed49024a'

# Base URLs for API calls
riot_base_url = 'https://europe.api.riotgames.com'
lol_base_url = 'https://eun1.api.riotgames.com'

# Rate limiting constants
requests_per_second = 20
requests_per_minute = 100

# Initialize time tracking for rate limiting
start_time = time.time()
requests_count = 0

# Example list of summoners
summoner_list = [
    {'gameName': 'acelosss', 'tagLine': '12345'}
]

# Date range for matches (timestamps in milliseconds)
start_date = int(datetime(2023, 9, 1).timestamp() * 1000)
end_date = int(datetime(2024, 9, 1).timestamp() * 1000)

# Initialize lists to hold the data
filtered_match_data = []

def handle_rate_limiting():
    global requests_count, start_time
    elapsed_time = time.time() - start_time

    if requests_count >= requests_per_second:
        time.sleep(1)
        requests_count = 0

    if elapsed_time < 60 and requests_count >= requests_per_minute:
        time.sleep(60 - elapsed_time)
        start_time = time.time()
        requests_count = 0

def get_puuid(game_name, tag_line):
    global requests_count
    url = f'{riot_base_url}/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print(f"Error fetching PUUID for {game_name}#{tag_line}: {response.status_code} {response.text}")
        return None

def get_match_history(puuid):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match history for PUUID {puuid}: {response.status_code} {response.text}")
        return []

def get_filtered_match_details(matchId):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/{matchId}?api_key={api_key}'  # Updated endpoint to get match details
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        match_data = response.json()

        # Check if the match falls within the specified date range
        game_creation = match_data['info']['gameCreation']
        if not (start_date <= game_creation <= end_date):
            return None

        # Extracting relevant data based on the given requirements
        for participant in match_data['info']['participants']:
            match_details = {
                'summonerName': participant['summonerName'],
                'championId': participant['championId'],
                'role': participant['teamPosition'],
                'kills': participant['kills'],
                'deaths': participant['deaths'],
                'assists': participant['assists'],
                'totalDamageDealt': participant['totalDamageDealtToChampions'],
                'totalDamageTaken': participant['totalDamageTaken'],
                'goldEarned': participant['goldEarned'],
                'totalMinionsKilled': participant['totalMinionsKilled'],
            }

            # Add team performance data (objectives)
            for team in match_data['info']['teams']:
                if participant['teamId'] == team['teamId']:
                    match_details.update({
                        'win': team['win'],
                        'dragons': team['objectives']['dragon']['kills'],
                        'barons': team['objectives']['baron']['kills'],
                        'turretsKilled': team['objectives']['tower']['kills']
                    })

            # Append the filtered data for each participant
            filtered_match_data.append(match_details)
    else:
        print(f"Error fetching match details for match ID {matchId}: {response.status_code} {response.text}")
        return None

# Process each summoner in the list
for summoner in summoner_list:
    puuid = get_puuid(summoner['gameName'], summoner['tagLine'])
    if puuid:
        matchIds = get_match_history(puuid)
        for matchId in matchIds:
            get_filtered_match_details(matchId)

# Convert the list to a DataFrame
filtered_match_df = pd.DataFrame(filtered_match_data)

# Display the DataFrame (for verification purposes)
print("\nFiltered Match Data:")
print(filtered_match_df.head())

# Save DataFrame to CSV
#filtered_match_df.to_csv('filtered_match_data.csv', index=False)

print("Data collection complete.")

Error fetching match details for match ID EUN1_3637196922: 404 {"status":{"message":"Data not found - match file not found","status_code":404}}
Error fetching match details for match ID EUN1_3634902506: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match details for match ID EUN1_3627636256: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}

Filtered Match Data:
     summonerName  championId     role  kills  deaths  assists  \
0  kutyadobalo300         126      TOP      6       8        1   
1      user825x43          62   JUNGLE      3       8        5   
2           Śhâcø          75   MIDDLE      3       8        4   
3  before sunrise         115   BOTTOM      1       1        5   
4       Positions         267  UTILITY      1       6        4   

   totalDamageDealt  totalDamageTaken  goldEarned  totalMinionsKilled    win  \
0             18814             21971       11341                 194  False   
1             15388      

In [13]:
filtered_match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   summonerName        371 non-null    object
 1   championId          371 non-null    int64 
 2   role                371 non-null    object
 3   kills               371 non-null    int64 
 4   deaths              371 non-null    int64 
 5   assists             371 non-null    int64 
 6   totalDamageDealt    371 non-null    int64 
 7   totalDamageTaken    371 non-null    int64 
 8   goldEarned          371 non-null    int64 
 9   totalMinionsKilled  371 non-null    int64 
 10  win                 371 non-null    bool  
 11  dragons             371 non-null    int64 
 12  barons              371 non-null    int64 
 13  turretsKilled       371 non-null    int64 
dtypes: bool(1), int64(11), object(2)
memory usage: 38.2+ KB


In [14]:
filtered_match_df.to_csv('Summoner_3_filtered_match_data.csv', index=False)

In [15]:
import requests
import time
import pandas as pd
from datetime import datetime

# Your Riot API key
api_key = 'RGAPI-25f46fad-7ece-43e8-a13d-1351ed49024a'

# Base URLs for API calls
riot_base_url = 'https://europe.api.riotgames.com'
lol_base_url = 'https://eun1.api.riotgames.com'

# Rate limiting constants
requests_per_second = 20
requests_per_minute = 100

# Initialize time tracking for rate limiting
start_time = time.time()
requests_count = 0

# Example list of summoners
summoner_list = [
    {'gameName': 'LunaTown', 'tagLine': 'EUNE'}
]

# Date range for matches (timestamps in milliseconds)
start_date = int(datetime(2023, 9, 1).timestamp() * 1000)
end_date = int(datetime(2024, 9, 1).timestamp() * 1000)

# Initialize lists to hold the data
filtered_match_data = []

def handle_rate_limiting():
    global requests_count, start_time
    elapsed_time = time.time() - start_time

    if requests_count >= requests_per_second:
        time.sleep(1)
        requests_count = 0

    if elapsed_time < 60 and requests_count >= requests_per_minute:
        time.sleep(60 - elapsed_time)
        start_time = time.time()
        requests_count = 0

def get_puuid(game_name, tag_line):
    global requests_count
    url = f'{riot_base_url}/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print(f"Error fetching PUUID for {game_name}#{tag_line}: {response.status_code} {response.text}")
        return None

def get_match_history(puuid):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match history for PUUID {puuid}: {response.status_code} {response.text}")
        return []

def get_filtered_match_details(matchId):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/{matchId}?api_key={api_key}'  # Updated endpoint to get match details
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        match_data = response.json()

        # Check if the match falls within the specified date range
        game_creation = match_data['info']['gameCreation']
        if not (start_date <= game_creation <= end_date):
            return None

        # Extracting relevant data based on the given requirements
        for participant in match_data['info']['participants']:
            match_details = {
                'summonerName': participant['summonerName'],
                'championId': participant['championId'],
                'role': participant['teamPosition'],
                'kills': participant['kills'],
                'deaths': participant['deaths'],
                'assists': participant['assists'],
                'totalDamageDealt': participant['totalDamageDealtToChampions'],
                'totalDamageTaken': participant['totalDamageTaken'],
                'goldEarned': participant['goldEarned'],
                'totalMinionsKilled': participant['totalMinionsKilled'],
            }

            # Add team performance data (objectives)
            for team in match_data['info']['teams']:
                if participant['teamId'] == team['teamId']:
                    match_details.update({
                        'win': team['win'],
                        'dragons': team['objectives']['dragon']['kills'],
                        'barons': team['objectives']['baron']['kills'],
                        'turretsKilled': team['objectives']['tower']['kills']
                    })

            # Append the filtered data for each participant
            filtered_match_data.append(match_details)
    else:
        print(f"Error fetching match details for match ID {matchId}: {response.status_code} {response.text}")
        return None

# Process each summoner in the list
for summoner in summoner_list:
    puuid = get_puuid(summoner['gameName'], summoner['tagLine'])
    if puuid:
        matchIds = get_match_history(puuid)
        for matchId in matchIds:
            get_filtered_match_details(matchId)

# Convert the list to a DataFrame
filtered_match_df = pd.DataFrame(filtered_match_data)

# Display the DataFrame (for verification purposes)
print("\nFiltered Match Data:")
print(filtered_match_df.head())

# Save DataFrame to CSV
#filtered_match_df.to_csv('filtered_match_data.csv', index=False)

print("Data collection complete.")

Error fetching match details for match ID EUN1_3645065127: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match details for match ID EUN1_3645029560: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}

Filtered Match Data:
       summonerName  championId role  kills  deaths  assists  \
0                           154           4       9       14   
1  KädetönMestari69          38           9      11       11   
2            murera          90           6       9       15   
3          LunaTown         360           5      13       14   
4       TuttiTonttu         497           0       9       21   

   totalDamageDealt  totalDamageTaken  goldEarned  totalMinionsKilled    win  \
0             17020             38897        9534                  17  False   
1             20436             25211       11044                  25  False   
2             17562             17494       10539                  41  False   
3             18555  

In [16]:
filtered_match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870 entries, 0 to 869
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   summonerName        870 non-null    object
 1   championId          870 non-null    int64 
 2   role                870 non-null    object
 3   kills               870 non-null    int64 
 4   deaths              870 non-null    int64 
 5   assists             870 non-null    int64 
 6   totalDamageDealt    870 non-null    int64 
 7   totalDamageTaken    870 non-null    int64 
 8   goldEarned          870 non-null    int64 
 9   totalMinionsKilled  870 non-null    int64 
 10  win                 870 non-null    bool  
 11  dragons             870 non-null    int64 
 12  barons              870 non-null    int64 
 13  turretsKilled       870 non-null    int64 
dtypes: bool(1), int64(11), object(2)
memory usage: 89.3+ KB


In [17]:
filtered_match_df.to_csv('Summoner_4_filtered_match_data.csv', index=False)

In [22]:
import requests
import time
import pandas as pd
from datetime import datetime

# Your Riot API key
api_key = 'RGAPI-25f46fad-7ece-43e8-a13d-1351ed49024a'

# Base URLs for API calls
riot_base_url = 'https://europe.api.riotgames.com'
lol_base_url = 'https://eun1.api.riotgames.com'

# Rate limiting constants
requests_per_second = 20
requests_per_minute = 100

# Initialize time tracking for rate limiting
start_time = time.time()
requests_count = 0

# Example list of summoners
summoner_list = [
    {'gameName': 'Snow', 'tagLine': 'Wht'}
]

# Date range for matches (timestamps in milliseconds)
start_date = int(datetime(2023, 9, 1).timestamp() * 1000)
end_date = int(datetime(2024, 9, 1).timestamp() * 1000)

# Initialize lists to hold the data
filtered_match_data = []

def handle_rate_limiting():
    global requests_count, start_time
    elapsed_time = time.time() - start_time

    if requests_count >= requests_per_second:
        time.sleep(1)
        requests_count = 0

    if elapsed_time < 60 and requests_count >= requests_per_minute:
        time.sleep(60 - elapsed_time)
        start_time = time.time()
        requests_count = 0

def get_puuid(game_name, tag_line):
    global requests_count
    url = f'{riot_base_url}/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print(f"Error fetching PUUID for {game_name}#{tag_line}: {response.status_code} {response.text}")
        return None

def get_match_history(puuid):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match history for PUUID {puuid}: {response.status_code} {response.text}")
        return []

def get_filtered_match_details(matchId):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/{matchId}?api_key={api_key}'  # Updated endpoint to get match details
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        match_data = response.json()

        # Check if the match falls within the specified date range
        game_creation = match_data['info']['gameCreation']
        if not (start_date <= game_creation <= end_date):
            return None

        # Extracting relevant data based on the given requirements
        for participant in match_data['info']['participants']:
            match_details = {
                'summonerName': participant['summonerName'],
                'championId': participant['championId'],
                'role': participant['teamPosition'],
                'kills': participant['kills'],
                'deaths': participant['deaths'],
                'assists': participant['assists'],
                'totalDamageDealt': participant['totalDamageDealtToChampions'],
                'totalDamageTaken': participant['totalDamageTaken'],
                'goldEarned': participant['goldEarned'],
                'totalMinionsKilled': participant['totalMinionsKilled'],
            }

            # Add team performance data (objectives)
            for team in match_data['info']['teams']:
                if participant['teamId'] == team['teamId']:
                    match_details.update({
                        'win': team['win'],
                        'dragons': team['objectives']['dragon']['kills'],
                        'barons': team['objectives']['baron']['kills'],
                        'turretsKilled': team['objectives']['tower']['kills']
                    })

            # Append the filtered data for each participant
            filtered_match_data.append(match_details)
    else:
        print(f"Error fetching match details for match ID {matchId}: {response.status_code} {response.text}")
        return None

# Process each summoner in the list
for summoner in summoner_list:
    puuid = get_puuid(summoner['gameName'], summoner['tagLine'])
    if puuid:
        matchIds = get_match_history(puuid)
        for matchId in matchIds:
            get_filtered_match_details(matchId)

# Convert the list to a DataFrame
filtered_match_df = pd.DataFrame(filtered_match_data)

# Display the DataFrame (for verification purposes)
print("\nFiltered Match Data:")
print(filtered_match_df.head())

# Save DataFrame to CSV
#filtered_match_df.to_csv('filtered_match_data.csv', index=False)

print("Data collection complete.")

Error fetching match details for match ID EUN1_3655250375: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match details for match ID EUN1_3655226318: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}

Filtered Match Data:
     summonerName  championId     role  kills  deaths  assists  \
0  Ummese Ngemuva          80      TOP      5       3        2   
1        DeltaSWE          30   JUNGLE     11       4        5   
2            Áxel         238   MIDDLE      4       0        1   
3           Oce4N          21   BOTTOM      3       3        8   
4            Mňau         111  UTILITY      1       5        8   

   totalDamageDealt  totalDamageTaken  goldEarned  totalMinionsKilled   win  \
0             13816             10423        6313                  92  True   
1             17555             13323        9185                  18  True   
2              6934              5448        8717                 127  True   
3            

In [23]:
filtered_match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630 entries, 0 to 629
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   summonerName        630 non-null    object
 1   championId          630 non-null    int64 
 2   role                630 non-null    object
 3   kills               630 non-null    int64 
 4   deaths              630 non-null    int64 
 5   assists             630 non-null    int64 
 6   totalDamageDealt    630 non-null    int64 
 7   totalDamageTaken    630 non-null    int64 
 8   goldEarned          630 non-null    int64 
 9   totalMinionsKilled  630 non-null    int64 
 10  win                 630 non-null    bool  
 11  dragons             630 non-null    int64 
 12  barons              630 non-null    int64 
 13  turretsKilled       630 non-null    int64 
dtypes: bool(1), int64(11), object(2)
memory usage: 64.7+ KB


In [24]:
filtered_match_df.to_csv('Summoner_5_filtered_match_data.csv', index=False)

In [25]:
import requests
import time
import pandas as pd
from datetime import datetime

# Your Riot API key
api_key = 'RGAPI-25f46fad-7ece-43e8-a13d-1351ed49024a'

# Base URLs for API calls
riot_base_url = 'https://europe.api.riotgames.com'
lol_base_url = 'https://eun1.api.riotgames.com'

# Rate limiting constants
requests_per_second = 20
requests_per_minute = 100

# Initialize time tracking for rate limiting
start_time = time.time()
requests_count = 0

# Example list of summoners
summoner_list = [
     {'gameName': 'Kemseptyni', 'tagLine': 'EUNE'}
]

# Date range for matches (timestamps in milliseconds)
start_date = int(datetime(2023, 9, 1).timestamp() * 1000)
end_date = int(datetime(2024, 9, 1).timestamp() * 1000)

# Initialize lists to hold the data
filtered_match_data = []

def handle_rate_limiting():
    global requests_count, start_time
    elapsed_time = time.time() - start_time

    if requests_count >= requests_per_second:
        time.sleep(1)
        requests_count = 0

    if elapsed_time < 60 and requests_count >= requests_per_minute:
        time.sleep(60 - elapsed_time)
        start_time = time.time()
        requests_count = 0

def get_puuid(game_name, tag_line):
    global requests_count
    url = f'{riot_base_url}/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print(f"Error fetching PUUID for {game_name}#{tag_line}: {response.status_code} {response.text}")
        return None

def get_match_history(puuid):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match history for PUUID {puuid}: {response.status_code} {response.text}")
        return []

def get_filtered_match_details(matchId):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/{matchId}?api_key={api_key}'  # Updated endpoint to get match details
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        match_data = response.json()

        # Check if the match falls within the specified date range
        game_creation = match_data['info']['gameCreation']
        if not (start_date <= game_creation <= end_date):
            return None

        # Extracting relevant data based on the given requirements
        for participant in match_data['info']['participants']:
            match_details = {
                'summonerName': participant['summonerName'],
                'championId': participant['championId'],
                'role': participant['teamPosition'],
                'kills': participant['kills'],
                'deaths': participant['deaths'],
                'assists': participant['assists'],
                'totalDamageDealt': participant['totalDamageDealtToChampions'],
                'totalDamageTaken': participant['totalDamageTaken'],
                'goldEarned': participant['goldEarned'],
                'totalMinionsKilled': participant['totalMinionsKilled'],
            }

            # Add team performance data (objectives)
            for team in match_data['info']['teams']:
                if participant['teamId'] == team['teamId']:
                    match_details.update({
                        'win': team['win'],
                        'dragons': team['objectives']['dragon']['kills'],
                        'barons': team['objectives']['baron']['kills'],
                        'turretsKilled': team['objectives']['tower']['kills']
                    })

            # Append the filtered data for each participant
            filtered_match_data.append(match_details)
    else:
        print(f"Error fetching match details for match ID {matchId}: {response.status_code} {response.text}")
        return None

# Process each summoner in the list
for summoner in summoner_list:
    puuid = get_puuid(summoner['gameName'], summoner['tagLine'])
    if puuid:
        matchIds = get_match_history(puuid)
        for matchId in matchIds:
            get_filtered_match_details(matchId)

# Convert the list to a DataFrame
filtered_match_df = pd.DataFrame(filtered_match_data)

# Display the DataFrame (for verification purposes)
print("\nFiltered Match Data:")
print(filtered_match_df.head())

# Save DataFrame to CSV
#filtered_match_df.to_csv('filtered_match_data.csv', index=False)

print("Data collection complete.")

Error fetching match details for match ID EUN1_3637429646: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match details for match ID EUN1_3637385696: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}

Filtered Match Data:
     summonerName  championId     role  kills  deaths  assists  \
0    PROJECT Pykë         266      TOP      6       4        6   
1    greater good         107   JUNGLE      7       2       12   
2     bury me low           4   MIDDLE     11       3        6   
3    Chef He3ShOt         429   BOTTOM     12       5        8   
4  Return of Alex         412  UTILITY      2       0       23   

   totalDamageDealt  totalDamageTaken  goldEarned  totalMinionsKilled   win  \
0             21046             21108        9106                 116  True   
1             20120             25089       13169                  28  True   
2             17131             13641       13146                 196  True   
3            

In [26]:
filtered_match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738 entries, 0 to 737
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   summonerName        738 non-null    object
 1   championId          738 non-null    int64 
 2   role                738 non-null    object
 3   kills               738 non-null    int64 
 4   deaths              738 non-null    int64 
 5   assists             738 non-null    int64 
 6   totalDamageDealt    738 non-null    int64 
 7   totalDamageTaken    738 non-null    int64 
 8   goldEarned          738 non-null    int64 
 9   totalMinionsKilled  738 non-null    int64 
 10  win                 738 non-null    bool  
 11  dragons             738 non-null    int64 
 12  barons              738 non-null    int64 
 13  turretsKilled       738 non-null    int64 
dtypes: bool(1), int64(11), object(2)
memory usage: 75.8+ KB


In [27]:
filtered_match_df.to_csv('Summoner_6_filtered_match_data.csv', index=False)

In [28]:
import requests
import time
import pandas as pd
from datetime import datetime

# Your Riot API key
api_key = 'RGAPI-25f46fad-7ece-43e8-a13d-1351ed49024a'

# Base URLs for API calls
riot_base_url = 'https://europe.api.riotgames.com'
lol_base_url = 'https://eun1.api.riotgames.com'

# Rate limiting constants
requests_per_second = 20
requests_per_minute = 100

# Initialize time tracking for rate limiting
start_time = time.time()
requests_count = 0

# Example list of summoners
summoner_list = [
     {'gameName': 'gypsy', 'tagLine': 'gyros'}
]

# Date range for matches (timestamps in milliseconds)
start_date = int(datetime(2023, 9, 1).timestamp() * 1000)
end_date = int(datetime(2024, 9, 1).timestamp() * 1000)

# Initialize lists to hold the data
filtered_match_data = []

def handle_rate_limiting():
    global requests_count, start_time
    elapsed_time = time.time() - start_time

    if requests_count >= requests_per_second:
        time.sleep(1)
        requests_count = 0

    if elapsed_time < 60 and requests_count >= requests_per_minute:
        time.sleep(60 - elapsed_time)
        start_time = time.time()
        requests_count = 0

def get_puuid(game_name, tag_line):
    global requests_count
    url = f'{riot_base_url}/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print(f"Error fetching PUUID for {game_name}#{tag_line}: {response.status_code} {response.text}")
        return None

def get_match_history(puuid):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match history for PUUID {puuid}: {response.status_code} {response.text}")
        return []

def get_filtered_match_details(matchId):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/{matchId}?api_key={api_key}'  # Updated endpoint to get match details
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        match_data = response.json()

        # Check if the match falls within the specified date range
        game_creation = match_data['info']['gameCreation']
        if not (start_date <= game_creation <= end_date):
            return None

        # Extracting relevant data based on the given requirements
        for participant in match_data['info']['participants']:
            match_details = {
                'summonerName': participant['summonerName'],
                'championId': participant['championId'],
                'role': participant['teamPosition'],
                'kills': participant['kills'],
                'deaths': participant['deaths'],
                'assists': participant['assists'],
                'totalDamageDealt': participant['totalDamageDealtToChampions'],
                'totalDamageTaken': participant['totalDamageTaken'],
                'goldEarned': participant['goldEarned'],
                'totalMinionsKilled': participant['totalMinionsKilled'],
            }

            # Add team performance data (objectives)
            for team in match_data['info']['teams']:
                if participant['teamId'] == team['teamId']:
                    match_details.update({
                        'win': team['win'],
                        'dragons': team['objectives']['dragon']['kills'],
                        'barons': team['objectives']['baron']['kills'],
                        'turretsKilled': team['objectives']['tower']['kills']
                    })

            # Append the filtered data for each participant
            filtered_match_data.append(match_details)
    else:
        print(f"Error fetching match details for match ID {matchId}: {response.status_code} {response.text}")
        return None

# Process each summoner in the list
for summoner in summoner_list:
    puuid = get_puuid(summoner['gameName'], summoner['tagLine'])
    if puuid:
        matchIds = get_match_history(puuid)
        for matchId in matchIds:
            get_filtered_match_details(matchId)

# Convert the list to a DataFrame
filtered_match_df = pd.DataFrame(filtered_match_data)

# Display the DataFrame (for verification purposes)
print("\nFiltered Match Data:")
print(filtered_match_df.head())

# Save DataFrame to CSV
#filtered_match_df.to_csv('filtered_match_data.csv', index=False)

print("Data collection complete.")

Error fetching match details for match ID EUN1_3641542500: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match details for match ID EUN1_3641535287: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}

Filtered Match Data:
    summonerName  championId role  kills  deaths  assists  totalDamageDealt  \
0          fapek         360          37      16       28             80889   
1        darknet         427           2      10       65             13127   
2       Yangeloo          64          13      13       42             33811   
3                         10          17      13       49             52528   
4  BEDROCKgrupka         555          16       9       38             26595   

   totalDamageTaken  goldEarned  totalMinionsKilled   win  dragons  barons  \
0             41922       22289                 101  True      0.0     0.0   
1             30789       13981                  16  True      0.0     0.0   
2             437

In [29]:
filtered_match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   summonerName        668 non-null    object 
 1   championId          668 non-null    int64  
 2   role                668 non-null    object 
 3   kills               668 non-null    int64  
 4   deaths              668 non-null    int64  
 5   assists             668 non-null    int64  
 6   totalDamageDealt    668 non-null    int64  
 7   totalDamageTaken    668 non-null    int64  
 8   goldEarned          668 non-null    int64  
 9   totalMinionsKilled  668 non-null    int64  
 10  win                 596 non-null    object 
 11  dragons             596 non-null    float64
 12  barons              596 non-null    float64
 13  turretsKilled       596 non-null    float64
dtypes: float64(3), int64(8), object(3)
memory usage: 73.2+ KB


In [30]:
filtered_match_df.to_csv('Summoner_7_filtered_match_data.csv', index=False)

In [31]:
import requests
import time
import pandas as pd
from datetime import datetime

# Your Riot API key
api_key = 'RGAPI-25f46fad-7ece-43e8-a13d-1351ed49024a'

# Base URLs for API calls
riot_base_url = 'https://europe.api.riotgames.com'
lol_base_url = 'https://eun1.api.riotgames.com'

# Rate limiting constants
requests_per_second = 20
requests_per_minute = 100

# Initialize time tracking for rate limiting
start_time = time.time()
requests_count = 0

# Example list of summoners
summoner_list = [
     {'gameName': 'Nicram', 'tagLine': 'EUNE'}
]

# Date range for matches (timestamps in milliseconds)
start_date = int(datetime(2023, 9, 1).timestamp() * 1000)
end_date = int(datetime(2024, 9, 1).timestamp() * 1000)

# Initialize lists to hold the data
filtered_match_data = []

def handle_rate_limiting():
    global requests_count, start_time
    elapsed_time = time.time() - start_time

    if requests_count >= requests_per_second:
        time.sleep(1)
        requests_count = 0

    if elapsed_time < 60 and requests_count >= requests_per_minute:
        time.sleep(60 - elapsed_time)
        start_time = time.time()
        requests_count = 0

def get_puuid(game_name, tag_line):
    global requests_count
    url = f'{riot_base_url}/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print(f"Error fetching PUUID for {game_name}#{tag_line}: {response.status_code} {response.text}")
        return None

def get_match_history(puuid):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match history for PUUID {puuid}: {response.status_code} {response.text}")
        return []

def get_filtered_match_details(matchId):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/{matchId}?api_key={api_key}'  # Updated endpoint to get match details
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        match_data = response.json()

        # Check if the match falls within the specified date range
        game_creation = match_data['info']['gameCreation']
        if not (start_date <= game_creation <= end_date):
            return None

        # Extracting relevant data based on the given requirements
        for participant in match_data['info']['participants']:
            match_details = {
                'summonerName': participant['summonerName'],
                'championId': participant['championId'],
                'role': participant['teamPosition'],
                'kills': participant['kills'],
                'deaths': participant['deaths'],
                'assists': participant['assists'],
                'totalDamageDealt': participant['totalDamageDealtToChampions'],
                'totalDamageTaken': participant['totalDamageTaken'],
                'goldEarned': participant['goldEarned'],
                'totalMinionsKilled': participant['totalMinionsKilled'],
            }

            # Add team performance data (objectives)
            for team in match_data['info']['teams']:
                if participant['teamId'] == team['teamId']:
                    match_details.update({
                        'win': team['win'],
                        'dragons': team['objectives']['dragon']['kills'],
                        'barons': team['objectives']['baron']['kills'],
                        'turretsKilled': team['objectives']['tower']['kills']
                    })

            # Append the filtered data for each participant
            filtered_match_data.append(match_details)
    else:
        print(f"Error fetching match details for match ID {matchId}: {response.status_code} {response.text}")
        return None

# Process each summoner in the list
for summoner in summoner_list:
    puuid = get_puuid(summoner['gameName'], summoner['tagLine'])
    if puuid:
        matchIds = get_match_history(puuid)
        for matchId in matchIds:
            get_filtered_match_details(matchId)

# Convert the list to a DataFrame
filtered_match_df = pd.DataFrame(filtered_match_data)

# Display the DataFrame (for verification purposes)
print("\nFiltered Match Data:")
print(filtered_match_df.head())

# Save DataFrame to CSV
#filtered_match_df.to_csv('filtered_match_data.csv', index=False)

print("Data collection complete.")

Error fetching match details for match ID EUN1_3643948039: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match details for match ID EUN1_3643251453: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}

Filtered Match Data:
       summonerName  championId     role  kills  deaths  assists  \
0           Prymo 2         246      TOP      3       6        4   
1  I LOVE SHALLTEAR         233   JUNGLE      3       9       12   
2   iLickBriarsFeet          55   MIDDLE      7       8        8   
3        WellHidden          21   BOTTOM      6       9        7   
4           Natrium         526  UTILITY      3       9       10   

   totalDamageDealt  totalDamageTaken  goldEarned  totalMinionsKilled    win  \
0             12017             15380        9186                 165  False   
1             12123             27242        8755                  12  False   
2             18736             20190        7962                  80  False 

In [32]:
filtered_match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 716 entries, 0 to 715
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   summonerName        716 non-null    object 
 1   championId          716 non-null    int64  
 2   role                716 non-null    object 
 3   kills               716 non-null    int64  
 4   deaths              716 non-null    int64  
 5   assists             716 non-null    int64  
 6   totalDamageDealt    716 non-null    int64  
 7   totalDamageTaken    716 non-null    int64  
 8   goldEarned          716 non-null    int64  
 9   totalMinionsKilled  716 non-null    int64  
 10  win                 708 non-null    object 
 11  dragons             708 non-null    float64
 12  barons              708 non-null    float64
 13  turretsKilled       708 non-null    float64
dtypes: float64(3), int64(8), object(3)
memory usage: 78.4+ KB


In [33]:
filtered_match_df.to_csv('Summoner_8_filtered_match_data.csv', index=False)

In [34]:
import requests
import time
import pandas as pd
from datetime import datetime

# Your Riot API key
api_key = 'RGAPI-25f46fad-7ece-43e8-a13d-1351ed49024a'

# Base URLs for API calls
riot_base_url = 'https://europe.api.riotgames.com'
lol_base_url = 'https://eun1.api.riotgames.com'

# Rate limiting constants
requests_per_second = 20
requests_per_minute = 100

# Initialize time tracking for rate limiting
start_time = time.time()
requests_count = 0

# Example list of summoners
summoner_list = [
     {'gameName': 'Marcel', 'tagLine': '1v9'}
]

# Date range for matches (timestamps in milliseconds)
start_date = int(datetime(2023, 9, 1).timestamp() * 1000)
end_date = int(datetime(2024, 9, 1).timestamp() * 1000)

# Initialize lists to hold the data
filtered_match_data = []

def handle_rate_limiting():
    global requests_count, start_time
    elapsed_time = time.time() - start_time

    if requests_count >= requests_per_second:
        time.sleep(1)
        requests_count = 0

    if elapsed_time < 60 and requests_count >= requests_per_minute:
        time.sleep(60 - elapsed_time)
        start_time = time.time()
        requests_count = 0

def get_puuid(game_name, tag_line):
    global requests_count
    url = f'{riot_base_url}/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print(f"Error fetching PUUID for {game_name}#{tag_line}: {response.status_code} {response.text}")
        return None

def get_match_history(puuid):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&api_key={api_key}'
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match history for PUUID {puuid}: {response.status_code} {response.text}")
        return []

def get_filtered_match_details(matchId):
    global requests_count
    url = f'{riot_base_url}/lol/match/v5/matches/{matchId}?api_key={api_key}'  # Updated endpoint to get match details
    response = requests.get(url)
    requests_count += 1
    handle_rate_limiting()

    if response.status_code == 200:
        match_data = response.json()

        # Check if the match falls within the specified date range
        game_creation = match_data['info']['gameCreation']
        if not (start_date <= game_creation <= end_date):
            return None

        # Extracting relevant data based on the given requirements
        for participant in match_data['info']['participants']:
            match_details = {
                'summonerName': participant['summonerName'],
                'championId': participant['championId'],
                'role': participant['teamPosition'],
                'kills': participant['kills'],
                'deaths': participant['deaths'],
                'assists': participant['assists'],
                'totalDamageDealt': participant['totalDamageDealtToChampions'],
                'totalDamageTaken': participant['totalDamageTaken'],
                'goldEarned': participant['goldEarned'],
                'totalMinionsKilled': participant['totalMinionsKilled'],
            }

            # Add team performance data (objectives)
            for team in match_data['info']['teams']:
                if participant['teamId'] == team['teamId']:
                    match_details.update({
                        'win': team['win'],
                        'dragons': team['objectives']['dragon']['kills'],
                        'barons': team['objectives']['baron']['kills'],
                        'turretsKilled': team['objectives']['tower']['kills']
                    })

            # Append the filtered data for each participant
            filtered_match_data.append(match_details)
    else:
        print(f"Error fetching match details for match ID {matchId}: {response.status_code} {response.text}")
        return None

# Process each summoner in the list
for summoner in summoner_list:
    puuid = get_puuid(summoner['gameName'], summoner['tagLine'])
    if puuid:
        matchIds = get_match_history(puuid)
        for matchId in matchIds:
            get_filtered_match_details(matchId)

# Convert the list to a DataFrame
filtered_match_df = pd.DataFrame(filtered_match_data)

# Display the DataFrame (for verification purposes)
print("\nFiltered Match Data:")
print(filtered_match_df.head())

# Save DataFrame to CSV
#filtered_match_df.to_csv('filtered_match_data.csv', index=False)

print("Data collection complete.")

Error fetching match details for match ID EUN1_3619038996: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match details for match ID EUN1_3618930914: 429 {"status":{"message":"Rate limit exceeded","status_code":429}}

Filtered Match Data:
       summonerName  championId     role  kills  deaths  assists  \
0            Dodosu         897      TOP      4       2        3   
1             Gochi          60   JUNGLE      5       3        7   
2  Hello There Eune         105   MIDDLE      7       2        8   
3     DontMakeMeMad         360   BOTTOM     12       5        4   
4   Daisy Whisperer         412  UTILITY      2       4       15   

   totalDamageDealt  totalDamageTaken  goldEarned  totalMinionsKilled   win  \
0             14451             19024       10415                 177  True   
1             11649             22613       11246                   9  True   
2             16528             15893       10721                 142  True   
3

In [35]:
filtered_match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808 entries, 0 to 807
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   summonerName        808 non-null    object 
 1   championId          808 non-null    int64  
 2   role                808 non-null    object 
 3   kills               808 non-null    int64  
 4   deaths              808 non-null    int64  
 5   assists             808 non-null    int64  
 6   totalDamageDealt    808 non-null    int64  
 7   totalDamageTaken    808 non-null    int64  
 8   goldEarned          808 non-null    int64  
 9   totalMinionsKilled  808 non-null    int64  
 10  win                 784 non-null    object 
 11  dragons             784 non-null    float64
 12  barons              784 non-null    float64
 13  turretsKilled       784 non-null    float64
dtypes: float64(3), int64(8), object(3)
memory usage: 88.5+ KB


In [36]:
filtered_match_df.to_csv('Summoner_9_filtered_match_data.csv', index=False)

In [37]:
import pandas as pd
import os

# List of CSV files to combine
csv_files = [
    'Summoner_1_filtered_match_data.csv', 'Summoner_2_filtered_match_data.csv', 'Summoner_3_filtered_match_data.csv', 'Summoner_4_filtered_match_data.csv', 
    'Summoner_5_filtered_match_data.csv', 'Summoner_6_filtered_match_data.csv', 'Summoner_7_filtered_match_data.csv', 'Summoner_8_filtered_match_data.csv', 'Summoner_9_filtered_match_data.csv'
]

# List to store DataFrames
dataframes = []

# Read each CSV and append it to the list
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('final_league_of_legends_data.csv', index=False)

print("Data combined successfully into 'final_league_of_legends_data.csv'.")


Data combined successfully into 'final_league_of_legends_data.csv'.
